# Testing Prospector Model & Likelihood

In [1]:
import os
import numpy as np
os.environ["SPS_HOME"] = '/global/homes/e/eramey16/fsps/'
h5_file = os.path.expandvars("$SCRATCH/data/monocle/test/10995462702105256.h5")

from docker import param_monocle as param
from docker import classify
import prospect.io.read_results as reader
import h5py
import pickle
import dynesty

In [2]:
res1, _, _ = reader.results_from(h5_file)

mag_unc_1 = np.array([0.005395851583585794, 0.0034580493543346622, 0.004425482546800575,
                 0.004152162899013791, 0.011291867461568727, 0.04210798998385776])*5
args_1 = {
    "mag_in": [20.717947, 19.274029, 18.668655, 18.314737, 17.77136, 18.470882],
    "mag_unc_in": mag_unc_1,
    "object_redshift": None,
}
run_params = param.run_params
obs_1, model_1, sps_1, noise_model_1 = param.load_all(args=args_1, **run_params)
spec_noise_1, phot_noise_1 = noise_model_1

Loaded observation
Loaded SPS libraries


In [3]:
mag_unc_2 = np.array([0.005395851583585794, 0.0034580493543346622, 0.004425482546800575,
                 0.004152162899013791, 0.011291867461568727, 0.04210798998385776])
args_2 = {
    "mag_in": [20.717947, 19.274029, 18.668655, 18.314737, 17.77136, 18.470882],
    "mag_unc_in": mag_unc_2,
    "object_redshift": None,
}

obs_2, model_2, sps_2, noise_model_2 = param.load_all(args=args_2, **run_params)
spec_noise_2, phot_noise_2 = noise_model_2

Loaded observation
Loaded SPS libraries


In [4]:
def loglikelihood1(theta_vec):
    return param.lnprobfn(theta_vec, model=model_1, obs=obs_1, def_sps=sps_1,
                          def_noise_model=noise_model_1)
def loglikelihood2(theta_vec):
    return param.lnprobfn(theta_vec, model=model_2, obs=obs_2, def_sps=sps_2, 
                          def_noise_model=noise_model_2)

In [5]:
loglikelihood1(res1['chain'][0,:]), loglikelihood2(res1['chain'][0,:])

(-261159996.25403342, -543201163.3229702)

In [6]:
with open('10995462702105256.pkl', 'rb') as file:
    dres = pickle.load(file)

In [9]:
N = dres['samples'].shape[0]
logl2 = [loglikelihood2(dres['samples'][i]) for i in range(N)]

In [10]:
dres_rwt = dynesty.utils.reweight_run(dres, logp_new=logl2)

AssertionError: blob

In [ ]:
len(dres['samples'][0])

In [ ]:
res1['chain']

# Tutorial on importance re-weighting

In [10]:
# system functions that are always useful to have
import time, sys, os

# basic numeric setup
import numpy as np
from numpy import linalg

# inline plotting
%matplotlib inline

# plotting
import matplotlib
from matplotlib import pyplot as plt

# seed the random number generator
rstate = np.random.default_rng(510)
import dynesty

In [11]:
ndim = 3  # number of dimensions
C = np.identity(ndim)  # set covariance to identity matrix
C[C==0] = 0.95  # set off-diagonal terms (strongly correlated)
Cinv = linalg.inv(C)  # precision matrix
lnorm = -0.5 * (np.log(2 * np.pi) * ndim + np.log(linalg.det(C)))  # ln(normalization)

# 3-D correlated multivariate normal log-likelihood
def loglikelihood(x):
    """Multivariate normal log-likelihood."""
    return -0.5 * np.dot(x, np.dot(Cinv, x)) + lnorm

In [12]:
# prior transform
def prior_transform(u):
    """Transforms our unit cube samples `u` to a flat prior between -10. and 10. in each variable."""
    return 10. * (2. * u - 1.)

In [13]:
# initialize our nested sampler
dsampler = dynesty.DynamicNestedSampler(loglikelihood, prior_transform, ndim=3,
                                        bound='single', sample='unif', rstate=rstate)
dsampler.run_nested(maxiter=500, use_stop=False)
dres = dsampler.results

iter: 50001 | batch: 22 | bound: 2 | nc: 1 | ncall: 877799 | eff(%):  5.696 | loglstar:   -inf < -0.407 < -1.059 | logz: -8.965 +/-  0.044 | stop:    nan                                             

In [18]:
import pickle

In [24]:
path = os.path.expandvars("$SCRATCH/data/monocle/test/deleteme_dynesty.dat")
with open(path, 'wb') as file:
    pickle.dump(dres, file)

In [25]:
with open(path, 'rb') as file:
    results_pkl = pickle.load(file)

In [17]:
dres.summary()

Summary
niter: 50001
ncall: 877799
eff(%):  5.696
logz: -8.923 +/-  0.043


In [16]:
dres.keys()

['niter',
 'ncall',
 'eff',
 'samples',
 'samples_id',
 'samples_batch',
 'samples_it',
 'samples_u',
 'samples_n',
 'logwt',
 'logl',
 'logvol',
 'logz',
 'batch_nlive',
 'batch_bounds',
 'logzerr',
 'information',
 'bound',
 'bound_iter',
 'samples_bound',
 'scale']

# Older stuff:

In [1]:
import os
os.environ["SPS_HOME"] = '/global/homes/e/eramey16/fsps/'

import fsps
import dynesty
import sedpy
import h5py, astropy
import numpy as np
import pandas as pd
from docker import classify

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
ls_id = 9906619143751091 #IPTF16geu

In [3]:
from prospect import prospect_args
parser = prospect_args.get_parser()
parser.print_help()

usage: ipykernel_launcher.py [-h] [--verbose VERBOSE] [--debug]
                             [--outfile OUTFILE] [--output_pickle]
                             [--zcontinuous ZCONTINUOUS] [--optimize]
                             [--min_method MIN_METHOD] [--min_opts MIN_OPTS]
                             [--nmin NMIN] [--emcee] [--nwalkers NWALKERS]
                             [--niter NITER] [--nburn [NBURN ...]]
                             [--save_interval INTERVAL]
                             [--restart_from RESTART_FROM]
                             [--ensemble_dispersion INITIAL_DISP] [--dynesty]
                             [--nested_bound {single,multi,balls,cubes}]
                             [--nested_sample {unif,rwalk,slice}]
                             [--nested_walks NESTED_WALKS]
                             [--nlive_init NESTED_NLIVE_INIT]
                             [--nlive_batch NESTED_NLIVE_BATCH]
                             [--nested_dlogz_init NESTED_DLOGZ_

In [4]:
# Try it with a more standard fit
from sedpy.observate import load_filters
from prospect.utils.obsutils import fix_obs

bkdata, tbldata = classify.get_galaxy(ls_id=ls_id)
gal_data = tbldata.iloc[0]
gal_data

ls_id            9906619143751091
ra                     316.066216
dec                     -6.340221
type                          REX
dered_mag_g             20.250755
dered_mag_r              18.86748
dered_mag_z             17.944523
dered_mag_w1             17.25038
dered_mag_w2            17.611616
dered_flux_g             7.937765
dered_flux_r             28.37973
dered_flux_z             66.40348
dered_flux_w1           125.84846
dered_flux_w2            90.23046
snr_g                   128.86171
snr_r                   213.08531
snr_z                   480.05804
snr_w1                  143.01906
snr_w2                   59.99879
flux_ivar_g              430.3113
flux_ivar_r              78.43762
flux_ivar_z             62.868927
flux_ivar_w1             1.328257
flux_ivar_w2             0.449847
dchisq_1                236241.66
dchisq_2                263783.94
dchisq_3                266028.38
dchisq_4                264630.44
dchisq_5                      0.0
rchisq_g      

In [5]:
# Get galaxy data
bands = ['g', 'r', 'z', 'w1', 'w2']
sdss = ['sdss_{}0'.format(b) for b in ['g','r','z']]
wise = ['wise_w1', 'wise_w2']
filternames = sdss + wise

filters = load_filters(filternames)
maggies = np.array([10**(-.4*gal_data[f'dered_mag_{b}']) for b in bands])
magerr = [ 2.5 / (np.log(10) * gal_data['dered_flux_'+b] * 
                        np.sqrt(gal_data['flux_ivar_'+b])) for b in bands]
magerr = np.clip(magerr, 0.05, np.inf)

In [6]:
# Build obs
obs = dict(wavelength=None, spectrum=None, unc=None, redshift=gal_data['z_phot_median'],
           maggies=maggies, maggies_unc=magerr * maggies / 1.086, filters=filters)
obs = fix_obs(obs)

In [7]:
# Build model
from prospect.models.templates import TemplateLibrary
from prospect.models import PolySpecModel
model_params = TemplateLibrary["continuity_sfh"]
model_params.update(TemplateLibrary["nebular"])
model_params["zred"]["init"] = obs["redshift"]

model = PolySpecModel(model_params)
noise_model = (None, None)

print(model)

:::::::
<class 'prospect.models.sedmodel.PolySpecModel'>

Free Parameters: (name: prior) 
-----------
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0)
  logmass: <class 'prospect.models.priors.TopHat'>(mini=7,maxi=12)
  logsfr_ratios: <class 'prospect.models.priors.StudentT'>(mean=[0. 0.],scale=[0.3 0.3],df=[2 2])

Fixed Parameters: (name: value [, depends_on]) 
-----------
  zred: [0.263183] 
  mass: [1000000.] <function logsfr_ratios_to_masses at 0x7f57687b7f70>
  sfh: [3] 
  imf_type: [2] 
  dust_type: [0] 
  agebins: [[ 0.  8.]
 [ 8.  9.]
 [ 9. 10.]] 
  add_neb_emission: [ True] 
  add_neb_continuum: [ True] 
  nebemlineinspec: [ True] 
  gas_logz: [0.] <function stellar_logzsol at 0x7f5768d96310>
  gas_logu: [-2.] 


In [8]:
from prospect.sources import CSPSpecBasis
sps = CSPSpecBasis(zcontinuous=1)
print(sps.ssp.libraries)

(b'mist', b'miles', b'DL07')


In [ ]:
from prospect.fitting import lnprobfn, fit_model
fitting_kwargs = dict(nlive_init=400, nested_method="rwalk", nested_target_n_effective=1000, nested_dlogz_init=0.05)
output = fit_model(obs, model, sps, optimize=True, dynesty=True, lnprobfn=lnprobfn, noise=noise_model, **fitting_kwargs)
result, duration = output["sampling"]

In [ ]:
from prospect.io import write_results as writer
hfile = "./quickstart_dynesty_mcmc.h5"
writer.write_hdf5(hfile, {}, model, obs,
                 output["sampling"][0], None,
                 sps=sps,
                 tsample=output["sampling"][1],
                 toptimize=0.0)